In [99]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px

import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

import re
import time
import os

In [100]:
def change_estimated_revenue_to_int(text):
    match = re.match(r'\$(\d+(?:\.\d+)?)([mkb]?)', text)
    if not match:
        raise ValueError('Invalid string format')
        
    value = float(match.group(1))
    suffix = match.group(2)

    if suffix == 'k':
        multiplier = 1000
    elif suffix == 'm':
        multiplier = 1000000
    elif suffix == 'b':
        multiplier = 1000000000
    else:
        multiplier = 1
        
    return int(value * multiplier)

def change_estimated_units_sold_to_int(text):
    match = re.match(r'(\d+(?:\.\d+)?)([mkb]?)', text)
    if not match:
        raise ValueError('Invalid string format')
        
    value = float(match.group(1))
    suffix = match.group(2)

    if suffix == 'k':
        multiplier = 1000
    elif suffix == 'm':
        multiplier = 1000000
    elif suffix == 'b':
        multiplier = 1000000000
    else:
        multiplier = 1
        
    return int(value * multiplier)

def change_number_of_reviews_to_int(text):
    review_number = re.sub(r',', '', text)
    return int(review_number)

def change_steam_rating_to_float(text):
    match = re.match(r'^([\d\.]+)%$', text)
    if not match:
        raise ValueError('Invalid percentage string format')

    return float(match.group(1)) / 100

In [101]:
df = pd.read_csv('cleaned_games.csv')
game_id_list = df['AppID'].tolist() # 58851
game_name_list = df['Name'].tolist()

# sample
# game_id_list = game_id_list[:5]
# game_name_list = game_name_list[:5]

name_in_vg_list = []
estimated_revenue_list = []
estimated_units_sold_list = []
number_of_reviews_list = []
steam_rating_list = []

checkpoint_interval = 1000  # Save the data every 1000 iterations.

In [102]:
webdriver_path = 'chromedriver.exe'
url = 'https://vginsights.com/games-database'

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')  # Required on Windows

browser = webdriver.Chrome(executable_path=webdriver_path, options=chrome_options)
browser.get(url)
html = browser.page_source
soupVG = BeautifulSoup(html, 'html.parser')

C:\Users\JongbeenSong\AppData\Local\Temp\ipykernel_37272\928016153.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=webdriver_path, options=chrome_options)


In [103]:
for i, game_name in enumerate(game_name_list):
    search_box_element = browser.find_element(By.ID, 'mat-input-1')
    search_box_element.clear()
    search_box_element.send_keys(game_name)
    time.sleep(2)

    try:
        name_in_vg_list_element = browser.find_element(By.XPATH, '/html/body/app-root/mat-sidenav-container/mat-sidenav-content/app-games-database/div/div/div[2]/div[2]/mat-card/mat-card-content/p-table/div/div[1]/table/tbody/tr[1]/td[2]')
        estimated_revenue_element = browser.find_element(By.XPATH, '/html/body/app-root/mat-sidenav-container/mat-sidenav-content/app-games-database/div/div/div[2]/div[2]/mat-card/mat-card-content/p-table/div/div[1]/table/tbody/tr[1]/td[5]')
        estimated_units_sold_element = browser.find_element(By.XPATH, '/html/body/app-root/mat-sidenav-container/mat-sidenav-content/app-games-database/div/div/div[2]/div[2]/mat-card/mat-card-content/p-table/div/div[1]/table/tbody/tr[1]/td[6]')
        number_of_reviews_element = browser.find_element(By.XPATH, '/html/body/app-root/mat-sidenav-container/mat-sidenav-content/app-games-database/div/div/div[2]/div[2]/mat-card/mat-card-content/p-table/div/div[1]/table/tbody/tr[1]/td[7]')
        steam_rating_element = browser.find_element(By.XPATH, '/html/body/app-root/mat-sidenav-container/mat-sidenav-content/app-games-database/div/div/div[2]/div[2]/mat-card/mat-card-content/p-table/div/div[1]/table/tbody/tr[1]/td[8]')
        
        name_in_vg = name_in_vg_list_element.text
        estimated_revenue = estimated_revenue_element.text
        estimated_units_sold = estimated_units_sold_element.text
        number_of_reviews = number_of_reviews_element.text
        steam_rating = steam_rating_element.text

        estimated_revenue = change_estimated_revenue_to_int(estimated_revenue)
        estimated_units_sold = change_estimated_units_sold_to_int(estimated_units_sold)
        number_of_reviews = change_number_of_reviews_to_int(number_of_reviews)
        steam_rating = change_steam_rating_to_float(steam_rating)

        name_in_vg_list.append(name_in_vg)
        estimated_revenue_list.append(estimated_revenue)
        estimated_units_sold_list.append(estimated_units_sold)
        number_of_reviews_list.append(number_of_reviews)
        steam_rating_list.append(steam_rating)
    
    except:
        name_in_vg_list.append('NULL')
        estimated_revenue_list.append('NULL')
        estimated_units_sold_list.append('NULL')
        number_of_reviews_list.append('NULL')
        steam_rating_list.append('NULL')
        
    if (i + 1) % checkpoint_interval == 0 or i == len(game_name_list) - 1:
        data = {
            'game_id': game_id_list[:i + 1],
            'game_name': game_name_list[:i + 1],
            'name_in_vg': name_in_vg_list,
            'estimated_revenue': estimated_revenue_list,
            'estimated_units_sold': estimated_units_sold_list,
            'number_of_reviews': number_of_reviews_list,
            'steam_rating': steam_rating_list
        }
        vg_df_temp = pd.DataFrame(data)
        temp_filename = f'vg_data_temp_{i + 1}.csv'
        vg_df_temp.to_csv(temp_filename, encoding="utf-8", mode="w", index=False)
        print(f"Checkpoint saved: {temp_filename}")


Checkpoint saved: vg_data_temp_1000.csv


WebDriverException: Message: unknown error: ChromeDriver only supports characters in the BMP
  (Session info: headless chrome=111.0.5563.147)
Stacktrace:
Backtrace:
	(No symbol) [0x00BEDCE3]
	(No symbol) [0x00B839D1]
	(No symbol) [0x00A94DA8]
	(No symbol) [0x00AE8084]
	(No symbol) [0x00ABBF2A]
	(No symbol) [0x00ABB830]
	(No symbol) [0x00ADAECC]
	(No symbol) [0x00AB8726]
	(No symbol) [0x00ADB224]
	(No symbol) [0x00AED57C]
	(No symbol) [0x00ADACC6]
	(No symbol) [0x00AB6F68]
	(No symbol) [0x00AB80CD]
	GetHandleVerifier [0x00E63832+2506274]
	GetHandleVerifier [0x00E99794+2727300]
	GetHandleVerifier [0x00E9E36C+2746716]
	GetHandleVerifier [0x00C96690+617600]
	(No symbol) [0x00B8C712]
	(No symbol) [0x00B91FF8]
	(No symbol) [0x00B920DB]
	(No symbol) [0x00B9C63B]
	BaseThreadInitThunk [0x76447D69+25]
	RtlInitializeExceptionChain [0x770EB74B+107]
	RtlClearBits [0x770EB6CF+191]


In [ ]:
data = {'game_id': game_id_list, 'game_name': game_name_list, 'name_in_vg': name_in_vg_list, 'estimated_revenue': estimated_revenue_list, 'estimated_units_sold': estimated_units_sold_list, 'number_of_reviews': number_of_reviews_list, 'steam_rating': steam_rating_list}
vg_df = pd.DataFrame(data)
vg_df.to_csv('vg_data.csv', encoding = "utf-8", mode = "w", index = False)